# Funciones

Código para recorrer los mejores dos casos de cada experimento y guardar sus resultados e hiperparámetros

In [ ]:
import os
import json
import pandas as pd
import numpy as np
from pandas import DataFrame

def open_results(results_path: str) -> DataFrame:
    """
    Lee el archivo CSV especificado en 'results_path', extrae métricas relevantes 
    y calcula el F1 score basado en precision y recall.

    Parámetros:
    -----------
    results_path : str
        Ruta al archivo 'progress.csv' que contiene los resultados del experimento.

    Retorna:
    --------
    DataFrame
        Un DataFrame con las columnas 'precision', 'recall', 'mAP50', 'mAP50-95' 
        y el F1 score calculado.
    """
    columnas_interes = ['metrics/precision(M)', 'metrics/recall(M)', 'metrics/mAP50(M)', 'metrics/mAP50-95(M)']
    df = pd.read_csv(results_path)
    # Extraer las columnas específicas y hacer una copia
    datos = df[columnas_interes].copy()
    # Renombrar las columnas eliminando 'metrics/'
    datos = datos.rename(columns=lambda col: col.replace('metrics/', ''))
    # Calcular el F1 score y añadirlo
    datos['F1_score'] = 2 * (datos['precision(M)'] * datos['recall(M)']) / (datos['precision(M)'] + datos['recall(M)'])
    return datos


def obtener_f1_maximo(datos: DataFrame) -> pd.Series:
    """
    Encuentra la fila con el valor máximo de F1 score y retorna todas las columnas 
    asociadas a esa fila.

    Parámetros:
    -----------
    datos : DataFrame
        DataFrame con los resultados de un experimento, incluyendo la columna F1_score.

    Retorna:
    --------
    pd.Series
        Una serie que contiene la fila con el mayor F1 score.
    """
    # Encontrar el índice de la fila con el valor máximo en la columna 'F1_score'
    idx_maximo = datos['F1_score'].idxmax()
    # Retornar todas las columnas asociadas a esa fila
    fila_maxima = datos.loc[idx_maximo]
    return fila_maxima


def retornar_mejores_dos_experimentos(directorio_experimentos: str) -> tuple:
    """
    Busca y retorna los dos experimentos con los mejores F1 scores en un directorio dado.

    Parámetros:
    -----------
    directorio_experimentos : str
        Ruta al directorio que contiene los experimentos.

    Retorna:
    --------
    tuple
        Un par de tuplas. La primera contiene los IDs de los dos mejores experimentos. 
        La segunda contiene las filas completas de métricas asociadas a los mejores puntajes.
    """
    puntajes = {}
    
    # Iterar sobre los experimentos en el directorio
    for experimento in os.listdir(directorio_experimentos):
        experimento_path = os.path.join(directorio_experimentos, experimento)
    
        if os.path.isdir(experimento_path):
            numero_id = int(experimento.split('_')[3])  # Extraer ID del experimento
            results_path = os.path.join(experimento_path, "progress.csv")
            results = open_results(results_path)
            best_fila = obtener_f1_maximo(results)  # Retorna la fila completa
            puntajes[numero_id] = best_fila  # Guardamos la fila completa
    
    # Ordenar los experimentos por el valor de 'F1_score' en las filas
    llaves_mayores = sorted(puntajes, key=lambda x: puntajes[x]['F1_score'], reverse=True)[:2]
    # Obtener las filas completas (puntajes) correspondientes a esas llaves
    puntajes_mejores = [puntajes[llave] for llave in llaves_mayores]
    return tuple(llaves_mayores), tuple(puntajes_mejores)


def obtener_mejores_experimentos(directorio_todos_los_experimentos: str, diccionario_raytune: dict):
    """
    Actualiza el diccionario de resultados de RayTune con los dos mejores experimentos para cada tune,
    añadiendo sus IDs y métricas.

    Parámetros:
    -----------
    directorio_todos_los_experimentos : str
        Ruta base que contiene los experimentos organizados por tune.
    diccionario_raytune : dict
        Diccionario con información de cada tune, que se actualizará con los mejores experimentos.

    Retorno:
    --------
    None
    """
    for tune_number, tune_dict in diccionario_raytune.items():
        path_experimentos = os.path.join(directorio_todos_los_experimentos, tune_number)
        if not os.path.isdir(path_experimentos):
            continue
        procesar_tune(path_experimentos, tune_dict)

def procesar_tune(path_experimentos: str, tune_dict: dict):
    """
    Procesa un directorio de experimentos para un tune específico, actualizando el diccionario de resultados
    con los dos mejores experimentos y sus métricas.

    Parámetros:
    -----------
    path_experimentos : str
        Ruta al directorio que contiene los experimentos para un tune específico.
    tune_dict : dict
        Diccionario con información de un tune, que se actualizará con los mejores experimentos.

    Retorno:
    --------
    None
    """
    for tune_date in os.listdir(path_experimentos):
        complete_path_experimentos = os.path.join(path_experimentos, tune_date)
        try:
            best_experimentos, best_puntaje = retornar_mejores_dos_experimentos(complete_path_experimentos)
            actualizar_diccionario_tune(tune_dict, best_experimentos, best_puntaje)
        except FileNotFoundError:
            continue

def actualizar_diccionario_tune(tune_dict: dict, best_experimentos: tuple, best_puntaje: tuple):
    """
    Actualiza el diccionario de resultados de un tune con los IDs de los mejores experimentos y sus métricas.

    Parámetros:
    -----------
    tune_dict : dict
        Diccionario con información de un tune, que se actualizará con los mejores experimentos.
    best_experimentos : tuple
        IDs de los dos mejores experimentos.
    best_puntaje : tuple
        Métricas de los dos mejores experimentos.

    Retorno:
    --------
    None
    """
    tune_dict.update(best=best_experimentos)
    tune_dict.update(score_1=best_puntaje[0], score_2=best_puntaje[1])


def obtener_config(directorio_todos_los_experimentos: str, diccionario_raytune: dict):
    """
    Extrae la configuración de los dos mejores experimentos (definidos por sus IDs en el diccionario de resultados)
    desde un directorio de experimentos y los añade al diccionario `diccionario_raytune`.
    
    Parámetros:
    -----------
    directorio_todos_los_experimentos : str
        La ruta base donde están almacenados los directorios con los resultados de cada "tune".
    diccionario_raytune : dict
        Diccionario que contiene la información sobre los experimentos organizados por "tune".
    
    Retorno:
    --------
    None
    """
    for tune_number, tune_dict in diccionario_raytune.items():
        path_experimentos = os.path.join(directorio_todos_los_experimentos, tune_number)
        if not os.path.isdir(path_experimentos):
            continue
        procesar_tune_para_config(path_experimentos, tune_dict)


def procesar_tune_para_config(path_experimentos: str, tune_dict: dict):
    """
    Procesa un directorio de experimentos para un tune específico y actualiza el diccionario de resultados
    con las configuraciones de los dos mejores experimentos.
    
    Parámetros:
    -----------
    path_experimentos : str
        Ruta al directorio que contiene los experimentos para un tune específico.
    tune_dict : dict
        Diccionario con información de un tune, que se actualizará con las configuraciones
        de los mejores experimentos.
    
    Retorno:
    --------
    None
    """
    for tune_date in os.listdir(path_experimentos):
        try:
            complete_path_experimentos = os.path.join(path_experimentos, tune_date)
            for experimento in os.listdir(complete_path_experimentos):
                procesar_experimento_para_config(complete_path_experimentos, experimento, tune_dict)
        except Exception as e:
            print(f"Error procesando {complete_path_experimentos}: {e}")
            continue

def procesar_experimento_para_config(complete_path_experimentos: str, experimento: str, tune_dict: dict):
    """
    Procesa un experimento específico dentro de un tune y actualiza el diccionario de resultados
    con la configuración si pertenece a los mejores experimentos.
    
    Parámetros:
    -----------
    complete_path_experimentos : str
        Ruta completa al directorio que contiene los experimentos para un tune específico.
    experimento : str
        Nombre del directorio del experimento.
    tune_dict : dict
        Diccionario con información de un tune, que se actualizará con la configuración
        del experimento si es uno de los mejores.
    
    Retorno:
    --------
    None
    """
    experimento_path = os.path.join(complete_path_experimentos, experimento)
    if not os.path.isdir(experimento_path):
        return

    try:
        numero_id = int(experimento.split('_')[3])
        ids = tune_dict.get('best', ())
        if numero_id in ids:
            cargar_y_actualizar_config(experimento_path, tune_dict, numero_id, ids)
    except IndexError as e:
        print(f"Error procesando experimento en {experimento_path}: {e}")
        return

def cargar_y_actualizar_config(experimento_path: str, tune_dict: dict, numero_id: int, ids: tuple):
    """
    Carga la configuración de un experimento específico y actualiza el diccionario de resultados
    si el ID del experimento es uno de los mejores.
    
    Parámetros:
    -----------
    experimento_path : str
        Ruta al directorio del experimento.
    tune_dict : dict
        Diccionario con información de un tune, que se actualizará con la configuración
        del experimento si es uno de los mejores.
    numero_id : int
        ID del experimento.
    ids : tuple
        IDs de los mejores experimentos.
    
    Retorno:
    --------
    None
    """
    params_path = os.path.join(experimento_path, "params.json")
    try:
        with open(params_path, 'r') as archivo:
            params_dict = json.load(archivo)
        if numero_id == ids[0]:
            tune_dict["config1"] = params_dict
        elif numero_id == ids[1]:
            tune_dict["config2"] = params_dict
    except FileNotFoundError:
        print(f"Archivo params.json no encontrado en {experimento_path}")
    except json.JSONDecodeError:
        print(f"Error de decodificación JSON en {params_path}")


def crear_dataframe_experimentos(resultados: dict[str, dict]) -> DataFrame:
    """
    Crea un DataFrame a partir de los resultados de experimentos, incluyendo las métricas 
    y los IDs de los mejores experimentos.

    Parámetros:
    -----------
    resultados : dict[str, dict]
        Diccionario que contiene los resultados de los experimentos para cada tune.

    Retorna:
    --------
    DataFrame
        DataFrame que contiene el nombre del tune, IDs de los mejores experimentos, y sus métricas.
    """
    filas = []

    for tune_number, datos in resultados.items():
        nombre = datos.get('name', tune_number)  # Usamos el nombre o el tune si no está 'name'
        # Obtenemos la tupla de IDs de la llave 'best', si existe, sino NaN
        experimento_ids = datos.get('best', (np.nan, np.nan))
        # Si tiene 'score_1', obtenemos las métricas, sino ponemos NaN
        score_1 = datos.get('score_1', pd.Series(dtype=float)).add_suffix("_1")
        score_2 = datos.get('score_2', pd.Series(dtype=float)).add_suffix("_2")
        # Crear un diccionario para almacenar el nombre, IDs y las métricas
        fila = {'name': nombre, 'Experimentos': experimento_ids}
        fila.update(score_1.to_dict())  # Añadimos las métricas de score_1 al diccionario
        fila.update(score_2.to_dict())  # Añadimos las métricas de score_2 al diccionario
        filas.append(fila)

    # Convertimos la lista de filas a un DataFrame de pandas
    df = pd.DataFrame(filas)
    # Rellenar las columnas vacías con NaN si no tienen datos
    df = df.replace({None: np.nan})
    return df


def filtrar_metricas(df: DataFrame) -> DataFrame:
    """
    Filtra un DataFrame para incluir únicamente las métricas relevantes ('F1_score', 'mAP50', 'mAP50-95'),
    además de las columnas 'name' y 'Experimentos'.

    Parámetros:
    -----------
    df : DataFrame
        DataFrame que contiene todas las métricas de los experimentos.

    Retorna:
    --------
    DataFrame
        DataFrame filtrado con las métricas relevantes.
    """
    # Seleccionar columnas que contienen 'F1_score', 'mAP50' o 'mAP50-95'
    columnas_interes = df.filter(like='F1_score_1').columns.tolist() + \
                       df.filter(like='mAP50(M)_1').columns.tolist() + \
                       df.filter(like='mAP50-95(M)_1').columns.tolist() + \
                        df.filter(like='F1_score_2').columns.tolist() + \
                       df.filter(like='mAP50(M)_2').columns.tolist() + \
                       df.filter(like='mAP50-95(M)_2').columns.tolist()
    # Filtrar el DataFrame para que contenga las métricas y las columnas 'name' y 'Experimentos'
    df_filtrado = df[['name', 'Experimentos'] + columnas_interes]
    
    return df_filtrado

Guardar los resultados en un JSON

In [ ]:
def convertir_series_a_dict(diccionario: dict) -> dict:
    """
    Convierte todas las pandas.Series en el diccionario a diccionarios nativos de Python.

    Parameters:
    -----------
    diccionario: dict
        Diccionario que contiene pandas.Series como partes de sus valores.

    Returns:
    --------
    dict
        Nuevo diccionario con pandas.Series convertidas a diccionarios.
    """
    for key, value in diccionario.items():
        if isinstance(value, pd.Series):
            diccionario[key] = value.to_dict()
        elif isinstance(value, dict):
            diccionario[key] = convertir_series_a_dict(value)
    return diccionario

def guardar_diccionario_como_json(diccionario: dict, archivo_path: str):
    """
    Guarda un diccionario en un archivo JSON después de convertir pandas.Series a diccionarios.

    Parameters:
    -----------
    diccionario : dict
        Diccionario que contiene la información de los experimentos organizados por tune.
    archivo_path : str
        Ruta donde se guardará el archivo JSON.
    
    Returns:
    --------
    None
    """
    diccionario_convertido = convertir_series_a_dict(diccionario)
    with open(archivo_path, 'w') as archivo:
        json.dump(diccionario_convertido, archivo, indent=4)

# Deepfish

In [1]:
resultados_raytune_deepfish = {
"tune":   {"name": "Deepfish_yolov8n-seg_AdamW"},
"tune2":  {"name": "Deepfish_yolov8n-seg_SGD"},
"tune3":  {"name": "Deepfish_LO_yolov8n-seg_AdamW"},
"tune4":  {"name": "Deepfish_LO_yolov8n-seg_SGD"},
"tune5":  {"name": "Deepfish_yolov8s-seg_AdamW"},
"tune6":  {"name": "Deepfish_yolov8s-seg_SGD"},
"tune7":  {"name": "Deepfish_LO_yolov8s-seg_AdamW"},
"tune8":  {"name": "Deepfish_LO_yolov8s-seg_SGD"},
"tune9":  {"name": "Deepfish_yolov8m-seg_AdamW"},
"tune10": {"name": "Deepfish_yolov8m-seg_SGD"},
"tune11": {"name": "Deepfish_LO_yolov8m-seg_AdamW"},
"tune12": {"name": "Deepfish_LO_yolov8m-seg_SGD"},
"tune13": {"name": "Deepfish_yolov8l-seg_AdamW"},
"tune14": {"name": "Deepfish_yolov8l-seg_SGD"},
"tune15": {"name": "Deepfish_LO_yolov8l-seg_AdamW"},
"tune16": {"name": "Deepfish_LO_yolov8l-seg_SGD"},
"tune17": {"name": "Deepfish_yolov8x-seg_AdamW"},
"tune18": {"name": "Deepfish_yolov8x-seg_SGD"},
"tune19": {"name": "Deepfish_LO_yolov8x-seg_AdamW"},
"tune20": {"name": "Deepfish_LO_yolov8x-seg_SGD"},
"tune21": {"name": "Deepfish_yolov9c-seg_AdamW"},
"tune22": {"name": "Deepfish_yolov9c-seg_SGD"},
"tune23": {"name": "Deepfish_LO_yolov9c-seg_AdamW"},
"tune24": {"name": "Deepfish_LO_yolov9c-seg_SGD"},
"tune25": {"name": "Deepfish_yolov9e-seg_AdamW"},
"tune26": {"name": "Deepfish_yolov9e-seg_SGD"},
"tune27": {"name": "Deepfish_LO_yolov9e-seg_AdamW"},
"tune28": {"name": "Deepfish_LO_yolov9e-seg_SGD"}
}

Mostrar resultados

In [25]:
obtener_mejores_experimentos("runs/detect", resultados_raytune_deepfish)
obtener_config("runs/detect", resultados_raytune_deepfish)
dataframe_resultados = crear_dataframe_experimentos(resultados_raytune_deepfish)
df = filtrar_metricas(dataframe_resultados)
df.index = range(1, len(df) + 1)
df

,name,Experimentos,F1_score_1,mAP50(M)_1,mAP50-95(M)_1,F1_score_2,mAP50(M)_2,mAP50-95(M)_2
1,Deepfish_yolov8n-seg_AdamW,"(7, 1)",0.973801,0.980280,0.793480,0.973264,0.974080,0.773310
2,Deepfish_yolov8n-seg_SGD,"(0, 14)",0.973858,0.981151,0.744352,0.973779,0.989280,0.731980
3,Deepfish_LO_yolov8n-seg_AdamW,"(1, 16)",0.967304,0.976192,0.771690,0.966654,0.964890,0.721740
4,Deepfish_LO_yolov8n-seg_SGD,"(3, 18)",0.979969,0.973320,0.746780,0.977824,0.974700,0.706960
5,Deepfish_yolov8s-seg_AdamW,"(3, 6)",0.992963,0.991320,0.784720,0.980175,0.979000,0.780300
6,Deepfish_yolov8s-seg_SGD,"(2, 8)",0.986763,0.985330,0.781130,0.978976,0.986560,0.729320
7,Deepfish_LO_yolov8s-seg_AdamW,"(1, 12)",0.990327,0.987390,0.797730,0.986099,0.984470,0.813850
8,Deepfish_LO_yolov8s-seg_SGD,"(0, 7)",0.990488,0.985707,0.785524,0.973359,0.969550,0.755630
9,Deepfish_yolov8m-seg_AdamW,"(0, 17)",0.975251,0.980680,0.770250,0.973208,0.978980,0.801610
10,Deepfish_yolov8m-seg_SGD,"(8, 1)",0.979175,0.976400,0.792070,0.973864,0.989370,0.805380


In [29]:
guardar_diccionario_como_json(resultados_raytune_deepfish, 'resultados_raytune_deepfish.json')

Leer los resultados y obtener nuevos rangos de hiperparámetros para entrenamientos con Tuner de ultralytics.

In [1]:
from utility_tuning import leer_resultados_raytune

ruta_json = 'resultados_raytune_deepfish.json'
nuevo_diccionario_hyper_params = leer_resultados_raytune(ruta_json)

In [2]:
nuevo_diccionario_hyper_params

{'tune': {'name': 'Deepfish_yolov8n-seg_AdamW',
  'config': {'lr0': (0.0005359315294799338, 0.0012612426851543789),
   'lrf': (0.011443411535723761, 0.15229290737227907),
   'momentum': (0.463689843183442, 0.7814355778465799),
   'weight_decay': (0.0004909686082999137, 0.0009184298166713585),
   'warmup_epochs': (2.084127910764866, 5.372266534355726),
   'warmup_momentum': (0.12126871755759236, 0.5689849682734504)}},
 'tune2': {'name': 'Deepfish_yolov8n-seg_SGD',
  'config': {'lr0': (0.00656617566350054, 0.011111041600846226),
   'lrf': (0.08724492986830709, 0.5271980613139581),
   'momentum': (0.47283038095032687, 1.0168474965317604),
   'weight_decay': (1.4613059289389025e-05, 0.0010802465288968892),
   'warmup_epochs': (1.908974635580806, 5.31890646383621),
   'warmup_momentum': (0.2502488956209207, 0.47887157777606576)}},
 'tune3': {'name': 'Deepfish_LO_yolov8n-seg_AdamW',
  'config': {'lr0': (0.0006428039685631077, 0.0014142311768348247),
   'lrf': (0.052451613104620265, 0.5223939

# Salmones